*********************************************************************************************************
# A Tour of Python 3  
version 1.0.1  
Authors: Phil Pfeiffer, Zack Bunch, and Feyisayo Oyeniyi  
East Tennessee State University  
Last updated June 2021  

Chapter 15: author Shayne Parker, ed. Phil Pfeiffer  
*********************************************************************************************************

# 15. Threads, Processes, Parallelism, and Concurrency  
 15.1 [Conceptual Overview](#Threads-Conceptual-Overview)  
 &ensp; 15.1.1 [Differentiating Concurrency and Parallelism](#Threads-Differentiating-Concurrency-and-Parallelism)  
 &ensp; 15.1.2 [Differentiating Threads and Processes](#Threads-Differentiating-Threads-and-Processes)  
 15.2 [Working with Threads](#Threads-Working-with-Threads)  
 15.3 [Working with Processes](#Threads-Working-with-Processes)  
 15.4 [Managing Access to Shared Resources](#Threads-Managing-Access-to-Shared-Resources)  
 &ensp; 15.4.1 [Race Conditions](#Threads-Race-Conditions)  
 &ensp; 15.4.2 [Mutexes and Semaphores](#Threads-Mutexes-and-Semaphores)  
 &ensp; 15.4.3 [Deadlock](#Threads-Deadlock)  
 &ensp; 15.4.4 [Locks and Semaphores in Lib/threading.py](#Threads-Locks-and-Semaphores-in-Lib-threading.py)  
 &ensp; 15.4.5 [Locks and Semaphores in Lib/multiprocessing.py](#Threads-Locks-and-Semaphores-in-Lib-multiprocessing.py)  
 15.5 [Concurrent.futures](#Threads-Concurrent-Futures)  
 &ensp; 15.5.1 [Executor](#Threads-Executor)  
 &ensp; 15.5.2 [ThreadPoolExecutor](#Threads-ThreadPoolExecutor)  
 &ensp; 15.5.3 [ProcessPoolExecutor](#Threads-ProcessPoolExecutor)  
 &ensp; 15.5.4 [Future Objects](#Threads-Future-Objects)  
 &ensp; 15.5.5 [Working with Concurrent Futures](#Threads-Working-with-Concurrent-Futures)  
 15.6 [Python Interpreter (CPython) Limitations](#Threads-Python-Interpreter-(CPython)-Limitations)  

## 15.1 Conceptual Overview <a name='Threads-Conceptual-Overview'></a>

### 15.1.1 Differentiating Concurrency and Parallelism <a name='Threads-Differentiating-Concurrency-and-Parallelism'></a>

Parallelism and concurrency are standard techniques for improving a system's performance and usability. Since both involve doing two or more things in a common time frame, these terms are often used interchangeably. The techniques, however, entail different ways of managing computations:  

- **Concurrency**: Making progress on more than one thing at a time. This can include alternating among the things being done; it does not necessarily imply doing them at the same time. 
- **Parallelism**: Executing more than one thing at the same time.  

To visualize the difference, consider two scenarios involving two painters at a common site. In one, the painters bring one brush, which they share. Regardless of whether this need to share the brush slows their work - at times, each may need to wait for one coat of paint to dry before applying a second - they cannot both paint at once. In the other, each has their own paint brush. While they may have to pause at times in their work, neither ever has to wait on the other to resume painting.

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 15.1.1.1:**

</span><span style='color:navy'>In the first of the two following markdown cells, tell which scenario is concurrency and which scenario is parallelism. Then, in the second, create your own example that illustrates the differences between concurrency and parallelism.</span>
***


***


***


### 15.1.2 Differentiating Threads and Processes <a name='Threads-Differentiating-Threads-and-Processes'></a>

As is true for concurrency and parallelism, the two basic strategies for implementing parallel and concurrent logic, threading and multiprocessing, are sometimes confused. While both strategies provide scaffolding for managing multiple computations, they do so in decidedly different ways:  
- *Threads* are paths of execution within a process.  
- *Processes* are individual programs that are in the process of executing. 

In short, systems house processes, which in turn, house threads. 
In systems that support multithreading and multiprocessing, users *can* organize computations as one or more cooperating process,  each of which has one or more cooperating threads. How a computation *should* be organized depends on its users' requirements for speed and computational resiliency.

- Characteristics of threads  
    - Threads depend on their parent process.
        - Threads, by default, share data with their parent process and therefore each other.
            - Contemporary thread APIs, as a rule, also allow threads to have private data areas.
        - As a rule, terminating a process terminates its threads.  
            - Python's thread API has a setting that allows threads to persist after process termination.  

    - Different operating systems use different strategies to manage thread executions.  
        - In some systems, processes are responsible for managing how and when their threads run.
        - In others, the operating system manages thread execution directly.
        - In both, when a thread blocks - i.e., suspends execution while waiting on another computation to complete or a resource to become available -           control passes to another thread or process, depending on the policies for scheduling.  
        
    - Advantages
        - Threads can be used to structure a program in a way that clarifies its design.  
        - Creating and terminating threads is fast, compared to creating and terminating processes.  
        - Switching between a process's threads is fast, compared to switching between processes.  
        - Exchanging data between a process's threads is fast, compared to exchanging data between processes.
        
    - Disadvantages
        - Threaded logic requires skill to write, due to the sharing of memory and other process resources.  
        - Threaded logic can be difficult to debug, due to the sharing of memory and other process resources.  
        
        
        
- Characteristics of processes
    - Processes, by default, have independent address spaces.
    
    - Processes are managed by the operating system.
    
    - Advantages  
        - Interactions among communicating processes can be easier to design than comparable interactions among communicating threads, due to
            - Operating systems support for process interaction, in the form of (e.g.) pipes and filters architectures  
            - Language support for process interaction, in the form of (e.g.) Ada's rendezvous statement  
        - Interactions among communicating processes can be easier to debug than comparable interactions among communicating threads,           when processes don't share memory.  
          
    - Disadvantages
       - Creating and terminating processes is slow, compared to creating and terminating threads.
       - Switching between processes is slow, compared to switching between a process's threads.
       - Exchanging data between processes is slow, compared to exchanging data between a process's threads.  

## 15.2 Working with Threads <a name='Threads-Working-with-Threads'></a>

The library for working with threads, Lib/threading.py, is an interface to Python's lower level `_thread` module. To create a thread, use `threading.Thread`:

 &ensp;&ensp;&ensp;&ensp; `threading.Thread(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)`
- *group* should be `None`; it's reserved for future extension when a ThreadGroup class is implemented
- *target* is the callable object invoked by the thread's `run()` method
- *name* is the thread's name
- *args* is a tuple containing arguments for *target*
- *kwargs* is a dictionary of keyword arguments for *target*
- *daemon* threads are a special case; if *daemon* is not `None`, the thread is a daemon thread

The `Thread` class manages an activity that runs in its own thread of control. Once a thread object is created, its activity must be started by calling the thread's `start()` method. Once started, a thread is considered *alive* until the `run()` method completes.  

The following is a list of selected thread object methods:  

- `start()` - Start a thread's activity  
- `run()` - Invokes the *target* argument from the thread constructor, which presumably causes the thread to run. Must be called after `start()`. 
- `join(timeout=None)` - blocks the calling thread until the thread whose `join()` was called is terminated.  When the *timeout* argument is present and not `None`, it should be a floating point number specifying a timeout in seconds.
- `is_alive()` - returns `True` iff the thread is alive.

The following is a list of selected thread object attributes:

- `name` - string used for identification purposes only. Multiple threads may have the same name.  
- `ident` - a nonzero integer. Its value has no direct meaning.   
- `native_id` - a non-negative integer that serves as a unique, system-wide identifier for this thread.
- `daemon` - a boolean that, if `True`, directs Python to allow the thread to continue running after its host program terminates. Must be set before `start()` is called. Inherited from the creating thread.

The entire Python program exits when the last non-daemon thread ceases execution.

The following is a partial list of Lib/threading functions characterizing the status of a program's set of threads:  

- `threading.enumerate()`- returns a list of all Thread objects currently alive.  
- `threading.active_count()` - returns the number of Thread objects currently alive. The returned count is equal to the length of the list returned by enumerate().  
- `threading.main_thread()` - returns the process’s main Thread object. In normal conditions, the main thread is the thread from which the Python interpreter was started.  
- `threading.current_thread()` - returns the current Thread object, corresponding to the caller's thread of control. If the caller's thread of control was not created through the threading module, a dummy thread object with limited functionality is returned.  
- `threading.get_ident()`- returns the current thread’s "thread identifier"  
- `threading.get_native_id()` - returns the native integral Thread ID of the current thread assigned by the kernel.  
- `threading.excepthook(args,/)` - handles uncaught exception raised by Thread.run().  

Lib/threading.py also defines `threading.TIMEOUT_MAX`. This constant is the maximum value allowed for the timeout parameter of blocking functions. Specifying a timeout greater than this value will raise an OverflowError.

The following examples illustrate the use of threads. Detailed information can be found in the official documentation [here](https://docs.python.org/3/library/threading.html).

In [0]:
# 15.2.a Creating a thread with default parameters
import threading

def thread_info(thread):
  print("Thread name:\t\t",thread.getName())
  print("Thread id:\t\t",thread.ident)
  print("Thread native id:\t",thread.native_id)
  print("Thread running?\t\t",thread.is_alive())
  print("Thread is daemon?\t",thread.isDaemon())
  print("Thread timeout:\t\t",threading.TIMEOUT_MAX)
  print()

thread = threading.Thread()
print("Thread created. Displaying info...")
thread_info(thread)

In [ ]:
# 15.2.b Creating a thread with a programmer-specified name
import threading

def thread_info(thread):
  print("Thread name:\t\t",thread.getName())
  print("Thread id:\t\t",thread.ident)
  print("Thread native id:\t",thread.native_id)
  print("Thread running?\t\t",thread.is_alive())
  print("Thread is daemon?\t",thread.isDaemon())
  print("Thread timeout:\t\t",threading.TIMEOUT_MAX)
  print()

thread = threading.Thread(name='Example Thread')
print("Thread created. Displaying info...")
thread_info(thread)

In [ ]:
# 15.2.c Manipulating a thread's attributes
import threading

def thread_info(thread):
  print("Thread name:\t\t",thread.getName())
  print("Thread id:\t\t",thread.ident)
  print("Thread native id:\t",thread.native_id)
  print("Thread running?\t\t",thread.is_alive())
  print("Thread is daemon?\t",thread.isDaemon())
  print("Thread timeout:\t\t",threading.TIMEOUT_MAX)
  print()

thread = threading.Thread(name='Example Thread')
print("Thread created. Displaying info...")
thread_info(thread)

thread.name = 'Modified Thread Name'
print("Thread modified. Displaying info...")
thread_info(thread)

In [ ]:
# 15.2.d Creating a local data area
import threading

def thread_info(thread):
  print("Thread name:\t\t",thread.getName())
  print("Thread id:\t\t",thread.ident)
  print("Thread native id:\t",thread.native_id)
  print("Thread running?\t\t",thread.is_alive())
  print("Thread is daemon?\t",thread.isDaemon())
  print("Thread timeout:\t\t",threading.TIMEOUT_MAX)
  print()

def show_local(p,val, val2):
  try:
    print("Original val: ",p.val)
    print("Original val2: ",p.val2)
  except AttributeError:
    print("This thread has nothing in its local data yet.")
  p.val = val
  p.val2 = val2
  print(p.val)
  print(p.val2)
  print()

mydata = threading.local()
mydata.val = 99
mydata.val2 = "snozberries"

print("Main thread val: ",mydata.val)
print("Main thread val2: ",mydata.val2)
print()

thread1 = threading.Thread(target=show_local,args=(mydata,3,"blueberries"))
thread2 = threading.Thread(target=show_local,args=(mydata,17,"blackberries"))
thread1.start()
thread2.start()

In [ ]:
# 15.2.e Creating and running a thread with a callable
import threading

def thread_info(thread):
  print("Thread name:\t\t",thread.getName())
  print("Thread id:\t\t",thread.ident)
  print("Thread native id:\t",thread.native_id)
  print("Thread running?\t\t",thread.is_alive())
  print("Thread is daemon?\t",thread.isDaemon())
  print("Thread timeout:\t\t",threading.TIMEOUT_MAX)
  print()

def example_callable(p):
  print("Hello from a callable. Your parameter is ",p)

thread = threading.Thread(target=example_callable,args=("Bologna",))
print("Thread created. Displaying info...")
print()
thread_info(thread)

thread.start()
print("Thread started. Displaying info...")
thread_info(thread)

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 15.2.1:**

</span><span style='color:navy' >In the following markdown cell, discuss the output from the last example. Is it what you would expect, looking at the code? If not, why do you think that is? Run the last example a few more times and observe the output. What happens?</span>
***


***


## 15.3 Working with Processes <a name='Threads-Working-with-Processes'></a>

The library for working with processes is Lib/multiprocessing.py. Lib/multiprocessing.py's API is similar to the `threading` module's API, which makes it easier to use both libraries. The following are a few examples of these similarities.

 &ensp;&ensp;&ensp;&ensp;`multiprocessing.Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)`

- `start()` - starts a process's activity  
- `run()` - invokes the *target* argument from the process constructor, which presumably causes the thread to run. Must be called after `start()`. 
- `join([timeout])` - if *timeout* is `None` (default), blocks until the process whose `join()` method is called terminates.  If *timeout* is a positive number, it blocks at most *timeout* seconds.  
    - A process can be joined many times.  
    - A process cannot join itself - this would cause deadlock (more on deadlock later).  
    - Do not attempt to join a process before it has started.  
- `is_alive()` - returns `True` iff the process is alive.
    
Processes have similar attributes as threads, as well, including `name`, `daemon`, and `pid`,  the process's operating-system-specified ID. Before the process is spawned, `pid` is `None`.  
 
The following methods and attributes are specific to the `multiprocessing` API.  

- `exitcode` - the exit code returned by a process's *child process* - i.e., a process that the current process has created.   This value is `None` if the child has not terminated.   A negative value, -N, indicates termination by signal N.
- `authkey` - When `multiprocessing` is initialized, the OS assigns to it a random byte string.    When a process object is created, it inherits the authentication key (the byte string) from its parent.    This value can be changed. 
- `terminate()` - requests the current process to terminate, using `TerminateProcess()` on Windows and `SIGTERM` on Unix.  Termination can be blocked using a signal handler, in order (e.g.) to allow the process to release its resources before exiting.  
    - Termination orphans descendant processes: i.e., relieves them of having to return an exit code to their parent process before being reclaimed.
    - Does not trigger exit handlers, finally clauses, etc.
- `kill()` - terminates the process. Uses `SIGKILL` on Unix instead of `SIGTERM`.
- `close()` - releases all resources associated with a process and closes the process. 

A `multiprocessing` module object that lacks a documented counterpart in the `threading` module,  `multiprocessing.Pool` supports methods for managing groups of processes, including methods for reusing terminated process objects.  This lack of a comparable thread pool object is due to Python reference implementation's lack of support for true thread-level parallelism.

Detailed information can be found in the official documentation [here](https://docs.python.org/3/library/multiprocessing.html). 

In [ ]:
# 15.3.a Creating a process with default parameters
import multiprocessing

def process_info(process):
  print("Process name:\t\t",process.name)
  print("Process id:\t\t",process.ident)
  print("Process authkey:\t",process.authkey)
  print("Process native id:\t",process.pid)
  print("Process running?\t",process.is_alive())
  print("Process is daemon?\t",process.daemon)
  print()

process = multiprocessing.Process()
print("Process created. Displaying info...")
process_info(process)

In [ ]:
# 15.3.b Creating a process with programmer-specified name
import multiprocessing

def process_info(process):
  print("Process name:\t\t",process.name)
  print("Process id:\t\t",process.ident)
  print("Process authkey:\t",process.authkey)
  print("Process native id:\t",process.pid)
  print("Process running?\t",process.is_alive())
  print("Process is daemon?\t",process.daemon)
  print()

process = multiprocessing.Process(name='Example Process')
print("Process created. Displaying info...")
process_info(process)

In [ ]:
# 15.3.c Manipulating a process's attributes
import multiprocessing
import os

def process_info(process):
  print("Process name:\t\t",process.name)
  print("Process id:\t\t",process.ident)
  print("Process authkey:\t",process.authkey)
  print("Process native id:\t",process.pid)
  print("Process running?\t",process.is_alive())
  print("Process is daemon?\t",process.daemon)
  print()

process = multiprocessing.Process(name='Example Process')
print("Process created. Displaying info...")
process_info(process)

process.name = 'Modified Process Name'
process.authkey = os.urandom(8)
print("Process created. Displaying info...")
process_info(process)

In [ ]:
# 15.3.d Creating and running a process with a callable
import subprocess, os

program_file_name = "test.py"
program_content = [
  "import multiprocessing\n",
  "\n",
  "def process_info(process):\n",
  "  print('Process name:\t\t',process.name)\n",
  "  print('Process id:\t\t',process.ident)\n",
  "  print('Process authkey:\t',process.authkey)\n",
  "  print('Process native id:\t',process.pid)\n",
  "  print('Process running?\t',process.is_alive())\n",
  "  print('Process is daemon?\t',process.daemon)\n",
  "  print()\n",
  "\n",
  "def example_callable(p):\n",
  "  print('Hello from the callable. Your parameter is ',p)\n",
  "\n",
  "if __name__ == '__main__':\n",
  "  process = multiprocessing.Process(target=example_callable,args=('Pineapple Pizza',))\n",
  "  print('Process created. Displaying info...')\n",
  "  process_info(process)\n",
  "  process.start()\n",
  "  print('Process started. Displaying info...')\n",
  "  process_info(process)\n",
  "  process.join()\n",
  "  process.close()\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.

CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output: ' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

In [ ]:
# 15.3.e Using Pool
import subprocess, os

program_file_name = "test.py"
program_content = [
  "from multiprocessing import Pool\n",
  "\n",
  "def example_callable(p):\n",
  "  print('Hello from the callable. Your parameter is ',p)\n",
  "\n",
  "if __name__ == '__main__':\n",
  "  with Pool(5) as pool:\n",
  "    pool.map(example_callable,[3])\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.
CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output: ' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

## 15.4 Managing Access to Shared Resources <a name='Threads-Managing-Access-to-Shared-Resources'></a>

Two major, recurring issues related to shared, concurrent and/or parallel access to computational resources are race conditions and deadlock. Both arise from attempts by multiple threads or processes to access the same resource(s) at the same time.

### 15.4.1 Race Conditions <a name='Threads-Race-Conditions'></a>

A *race condition* is a scenario where a computation's outcome varies in an unpredictable way due to interference from other, concurrently executing computations. Race conditions usually involve an attempt by at least one computation to alter content at the same time that another computation is attempting to read or write that content.  

A classic example of a race condition involving a database involves an attempt to withdraw \\$10 from a \\$100 bank account while another person is depositing \\$10 to the same account. Consider the following scenario:  
- The program that processes the deposit determines that the account currently holds \\$100.  
- This program temporarily gives way to the other program, which deducts \\$10 from the account, leaving \\$90 in the account.  
- The first program then adds \\$10 to \\$100 and updates the balance to \\$110.  

The net result is that the account has \\$10 more in it than it should.  Reversing this scenario would leave the individual \\$10 poorer rather than richer.  

The common method of preventing race conditions is to implement a mechanism that allows a process or thread to maintain control over a resource until a sequence of atomic actions - actions that must execute as a unit - are complete. The two most common such mechanisms are mutexes and semaphores.

### 15.4.2 Mutexes and Semaphores <a name='Threads-Mutexes-and-Semaphores'></a>

Mutexes and semaphores are similar, but not identical. As seems to be common when discussing concurrency and parallelism, these terms are often mistakenly used interchangeably.

A **mutex** (**mut**ual **ex**clusion) is a locking mechanism. When a process or thread *claims* the mutex, it locks access to that data. When the mutex's owner is done with the resource, it *releases* the mutex, which allows another process or thread to claim the mutex.  

A **semaphore** is a signaling mechanism. There are two kinds of semaphores: 
-  A counting semaphore initializes, typically, to the amount of the resource that is available:    e.g., if a system has four cores, the semaphore is initialized to four.  
-  A binary semaphore initializes to one.  

Both kinds of semaphores operate in the same manner.  
- When access is granted, the semaphore is decremented by one.  
- When the thread or process is done, the semaphore is incremented by one.  

The decrement/increment operations *signal* a resource's availability. If the semaphore is at zero and a request for access comes in, the request is stored in a buffer. When a resource is released, the increment signals that something from the buffer can be popped off and given access. 

Mutexes and semaphores help control access to prevent race conditions. However, they create a second issue.

### 15.4.3 Deadlock <a name='Threads-Deadlock'></a>

To visualize deadlock, consider the earlier example of the two painters. Imagine that the one painter has the paint brush and the other has the can of paint. The first painter needs the paint can to continue painting while the second needs the brush.  Neither painter surrenders their respective resource. This is deadlock.

Addressing deadlock usually involves reconsidering how access to the resource is granted.

### 15.4.4 Locks and Semaphores in Lib/threading.py <a name='Threads-Locks-and-Semaphores-in-Lib-threading.py'></a>

The `threading` module provides several objects for coordinating access to shared resources.  

- `threading.Lock` - a non-recursive mechanism for asserting ownership over a shared resource.
    - Once a thread has acquired this lock, any other thread attempting to acquire it will block until the lock is released.  
    
    - `acquire(blocking=True, timeout=-1)` - attempt to acquire the lock, in a manner specified by the routine's two parameters
        - If *blocking* is `False`, do not block. If a call where *blocking* set to `True` would block, return `False` immediately.
        - If *timeout* is -1, it's an unbounded wait.
        
    - `release()`
        - Any thread can call this for any Lock. As a rule, however, a lock should only be released by the thread that last acquired it.  
        - If called for a lock that is not locked, raises a RuntimeError.
    
    
- `threading.RLock` - a reentrant object that allows a thread to repeatedly acquire the lock.  
    - The implementation uses the concepts of thread ownership and recursion levels to balance locking and unlocking operations.  
    
    - `acquire(blocking=True, timeout=-1)`  
        - If the thread already owns the lock, increment the recursion level.  
        - If another thread owns the lock, block until the lock is unlocked.  
        
    -`release()`  
        - Decrements the recursion level. If the recursion level is zero after the decrement, the lock is set to unlocked.
        

- `threading.Semaphore(value=1)`
    - Raises a ValueError if *value* is passed as a negative number.
    - `acquire(blocking=True, timeout=None)`
    - `release(n=1)`
        - Increment the semaphores internal counter by *n*. Wakes up *n* blocking threads if the internal counter was zero on entry.
        

- `threading.BoundedSemaphore(value=1)`
    - A bounded semaphore sets hard limits at 0 and *value*. If the internal counter exceeds *value*, it raises a ValueError.

In [ ]:
# 15.4.4.a Creating and using a Lock
import subprocess, os

program_file_name = "test.py"
program_content = [
  "import threading\n",
  "import time\n",
  "def thread_info(thread):\n",
  "  print('Thread name:\t\t',thread.getName())\n",
  "  print('Thread id:\t\t',thread.ident)\n",
  "  print('Thread native id:\t',thread.native_id)\n",
  "  print('Thread running?\t\t',thread.is_alive())\n",
  "  print('Thread is daemon?\t',thread.isDaemon())\n",
  "  print('Thread timeout:\t\t',threading.TIMEOUT_MAX)\n",
  "  print()\n",
  "def increment(val):\n",
  "  lock = threading.Lock()\n",
  "  lock.acquire()\n",
  "  try:\n",
  "    time.sleep(2)\n",
  "    val += 1\n",
  "    print(val)\n",
  "  finally:\n",
  "    lock.release()\n",
  "  return val\n",
"\n",
"if __name__ == '__main__':\n",
"  data = 10\n",
"  thread1 = threading.Thread(target=increment,args=(data,))\n",
"  thread1.name = 'Thread 1'\n",
"  thread2 = threading.Thread(target=increment,args=(data,))    \n",
"  thread2.name = 'Thread 2'\n",
"  thread1.start()\n",
"  thread_info(thread1)\n",
"  thread2.start()\n",
"  thread_info(thread2)\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.

CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output:\n' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

In [ ]:
# 15.4.4.b Creating and using a Lock using the 'with' statement
# 'with' releases the lock it acquires upon exit from the block

import subprocess, os

program_file_name = "test.py"
program_content = [
  "import threading\n",
  "import time\n",
  "def thread_info(thread):\n",
  "  print('Thread name:\t\t',thread.getName())\n",
  "  print('Thread id:\t\t',thread.ident)\n",
  "  print('Thread native id:\t',thread.native_id)\n",
  "  print('Thread running?\t\t',thread.is_alive())\n",
  "  print('Thread is daemon?\t',thread.isDaemon())\n",
  "  print('Thread timeout:\t\t',threading.TIMEOUT_MAX)\n",
  "  print()\n",
  "def increment(val):\n",
  "  lock = threading.Lock()\n",
  "  with lock:\n",
  "    time.sleep(2)\n",
  "    val += 1\n",
  "    print(val)\n",
  "  return val\n",
"\n",
"if __name__ == '__main__':\n",
"  data = 10\n",
"  thread1 = threading.Thread(target=increment,args=(data,))\n",
"  thread1.name = 'Thread 1'\n",
"  thread2 = threading.Thread(target=increment,args=(data,))    \n",
"  thread2.name = 'Thread 2'\n",
"  thread1.start()\n",
"  thread_info(thread1)\n",
"  thread2.start()\n",
"  thread_info(thread2)\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.

CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output:\n' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

In [ ]:
# 15.4.4.c Showing any thread can release a lock, even if not the owning thread
import subprocess, os

program_file_name = "test.py"
program_content = [
  "import threading\n",
  "import time\n",
  "def thread_info(thread):\n",
  "  print('Thread name:\t\t',thread.getName())\n",
  "  print('Thread id:\t\t',thread.ident)\n",
  "  print('Thread native id:\t',thread.native_id)\n",
  "  print('Thread running?\t\t',thread.is_alive())\n",
  "  print('Thread is daemon?\t',thread.isDaemon())\n",
  "  print('Thread timeout:\t\t',threading.TIMEOUT_MAX)\n",
  "  print()\n",
"\n",
"def example1(lock):\n",
"  print('Is this lock, locked? ', lock.locked())\n",
"  lock.acquire()\n",
"  print('Is this lock, locked? ', lock.locked())\n",
"\n",
"def example2(lock):\n",
"  print('Is this lock, locked? ', lock.locked())\n",
"  lock.release()\n",
"  print('Is this lock, locked? ', lock.locked())\n",
"\n",
"if __name__ == '__main__':\n",
"  lock = threading.Lock()\n",    
"  thread1 = threading.Thread(target=example1,args=(lock,))\n",
"  thread2 = threading.Thread(target=example2,args=(lock,))\n",
"  thread1.start()\n",
"  thread2.start()\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.

CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output:\n' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

In [ ]:
# 15.4.4.d Creating and using a Semaphore
import subprocess, os

program_file_name = "test.py"
program_content = [
  "import threading\n",
  "import time\n",
  "def thread_info(thread):\n",
  "  print('Thread name:\t\t',thread.getName())\n",
  "  print('Thread id:\t\t',thread.ident)\n",
  "  print('Thread native id:\t',thread.native_id)\n",
  "  print('Thread running?\t\t',thread.is_alive())\n",
  "  print('Thread is daemon?\t',thread.isDaemon())\n",
  "  print('Thread timeout:\t\t',threading.TIMEOUT_MAX)\n",
  "  print()\n",
  "def increment(val):\n",
  "  semaphore = threading.Semaphore()\n",
  "  semaphore.acquire()\n",
  "  try:\n",
  "    time.sleep(2)\n",
  "    val += 1\n",
  "    print(val)\n",
  "  finally:\n",
  "    semaphore.release()\n",
  "  return val\n",
"\n",
"if __name__ == '__main__':\n",
"  data = 10\n",
"  thread1 = threading.Thread(target=increment,args=(data,))\n",
"  thread1.name = 'Thread 1'\n",
"  thread2 = threading.Thread(target=increment,args=(data,))    \n",
"  thread2.name = 'Thread 2'\n",
"  thread1.start()\n",
"  thread_info(thread1)\n",
"  thread2.start()\n",
"  thread_info(thread2)\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.

CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output:\n' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

In [ ]:
# 15.4.4.e Creating and using a Semaphore using the 'with' statement
# 'with' releases the semaphore it acquires upon exit from the block

import subprocess, os

program_file_name = "test.py"
program_content = [
  "import threading\n",
  "import time\n",
  "def thread_info(thread):\n",
  "  print('Thread name:\t\t',thread.getName())\n",
  "  print('Thread id:\t\t',thread.ident)\n",
  "  print('Thread native id:\t',thread.native_id)\n",
  "  print('Thread running?\t\t',thread.is_alive())\n",
  "  print('Thread is daemon?\t',thread.isDaemon())\n",
  "  print('Thread timeout:\t\t',threading.TIMEOUT_MAX)\n",
  "  print()\n",
  "def increment(val):\n",
  "  semaphore = threading.Semaphore()\n",
  "  with semaphore:\n",
  "    time.sleep(2)\n",
  "    val += 1\n",
  "    print(val)\n",
  "  return val\n",
"\n",
"if __name__ == '__main__':\n",
"  data = 10\n",
"  thread1 = threading.Thread(target=increment,args=(data,))\n",
"  thread1.name = 'Thread 1'\n",
"  thread2 = threading.Thread(target=increment,args=(data,))    \n",
"  thread2.name = 'Thread 2'\n",
"  thread1.start()\n",
"  thread_info(thread1)\n",
"  thread2.start()\n",
"  thread_info(thread2)\n"
]

# The following code writes the above text strings as a file that is then executed.
# This is necessary to get output from the process.
# This code is sourced from Dr. Pfeiffer, from other Tour of Python units, and has been modified to suit this example.

CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure
# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  print( '>Executing<\n')
  program_status = subprocess.run( [ 'python', program_file_name ], capture_output=True )
  print( '>return code is', program_status.returncode,"<\n" )
  standard_output = byte_seq_to_string( program_status.stdout )
  print( '> no standard output returned <' if standard_output == '' else 'standard output:\n' + standard_output )
  error_output = byte_seq_to_string( program_status.stderr )
  print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
  os.remove( program_file_name )

### 15.4.5 Locks and Semaphores in Lib/multiprocessing.py <a name='Threads-Locks-and-Semaphores-in-Lib-multiprocessing.py'></a>

The `multiprocessing` module provides several classes that implement locks and semaphores. These classes are similar to those provided by the threading API, with a number of small differences.  

- `multiprocessing.Lock` - a non-recursive mechanism for asserting ownership over a shared resource.  
    - Once a process has acquired a lock, any other process attempting to acquire it will block until the lock is released.  
    - `acquire(block=True, timeout=None)`
        - If *block* is `False`, do not block. If a call where *block* set to `True` would block, return `False` immediately.           Note that in `threading` this is *blocking* but for `multiprocessing` it's *block*.
        - If *timeout* is `None`, it's an unbounded wait.           Note that in `threading` the default value is -1, but for `multiprocessing` it's `None`.  
    - `release()`
        - Any process can call this routine for any Lock.           As a rule, however, a lock should only be released by the process that last acquired it.  
        - If called for a lock that is not locked, raises a ValueError. Note that in `threading` this raises a RuntimeError instead.  


- `multiprocessing.RLock` - a reentrant object that allows a process to repeatedly acquire the lock.  
    - `acquire(block=True, timeout=None)`  
        - In `threading` this is *blocking* but for `multiprocessing` it's *block*.
        - In `threading` the default value for *timeout* is -1, but for `multiprocessing` it's `None`.  
    -`release()` - Raise an AssertionError if called by any process that does not own the lock.


- `multiprocessing.Semaphore([value])`
- `multiprocessing.BoundedSemaphore([value])`
    - The *value* argument is optional and has no default value for `multiprocessing` semaphores.

The examples for `threading` work for `multiprocessing` since the code is nearly identical.

## 15.5 Concurrent.futures <a name='Threads-Concurrent-Futures'></a>

`Concurrent.futures` provides a high-level interface for supporting the asynchronous execution of callable objects. Calling a future invokes a callable object and returns a handle on a second object for managing the callable’s execution. This includes checking to see if the callable has finished executing; waiting for the callable to finish and obtaining its result; and cancelling the callable’s execution, if the callable has not yet been started.  
Detailed information can be found in the [Python library documentation](https://docs.python.org/3/library/concurrent.futures.html).

`Concurrent.futures` provides two functions for monitoring the status of a computation's futures.

- `wait(fs, timeout=None, return_when=ALL_COMPLETED)` - waits for Future instances given by *fs* to complete, returning a named 2-tuple of sets.  
    - The first set is named `done`. It contains the futures that completed (i.e., cancelled or finished) before the `wait` completed.
    - The second set is named `not_done`. It contains the futures that did not complete (i.e., pending or running).
    - *return_when* indicates when `wait()` returns.
        - `FIRST_COMPLETED` returns when any future completes.
        - `FIRST_EXCEPTION` returns when any future raises an exception. If no exception is raised, this is equivalent to the next option.
        - `ALL_COMPLETED` returns when all futures complete.
- `as_completed(fs,timeout=None)` - returns an iterator over the *fs* futures as they complete. 

### 15.5.1 Executor <a name='Threads-Executor'></a>

`Concurrent.futures` provides an abstract class, `concurrent.futures.Executor`, which serves as a base class for instantiating futures. `Executor` defines three methods for its derived classes to implement.  

- `submit(fn, /, *args, **kwargs)` - schedules *fn* to be executed using *args* and *kwargs*.
    - Returns a `Future` object representing the callable execution.
- `map(func, *iterables, timeout=None, chunksize=1)` - returns an iterator that applied *func* to instance of *iterables*.  
    - *func* is executed asynchronously; multiple calls to *fun* may be executed concurrently.
    - *iterables* are launched immediately, not lazily.  
    - Returns an iterator.
        - If the result isn't available *timeout* seconds after the original call to `map()`,           this iterator raises a `concurrent.futures.TimeoutError` when `__next__()` is called.
        - If *func* raises an error, this error is not raised until it's accessed through the iterator.
    - *chunksize* specifies the size of a chunk used by the `ProcessPoolExecutor` concretion's iterator.       There is no effect for the `ThreadPoolExecutor` concretion.
- `shutdown(wait=True, *, cancel_futures=False)` - signals the executor to free any resources when currently pending futures are done executing.
    - If *wait* is `True`, `shutdown()` does not return until all futures finish and resources freed.       If `False`, `shutdown()` returns as soon as pending futures finish. 
    - If *cancel_futures* is `True`, cancel all pending futures that have not started running.
    - Using a `with` statement avoids having to call `shutdown()` explicitly. 

### 15.5.2 ThreadPoolExecutor <a name='Threads-ThreadPoolExecutor'></a>

`concurrent.futures.ThreadPoolExecutor` is a subclass of `Executor` that uses a thread pool for creating and managing futures.  

 &ensp;&ensp;&ensp;&ensp; `concurrent.futures.ThreadPoolExecutor(max_workers=None, thread_name_prefix='', initializer=None, initargs=())`  
- *max_workers* indicates the number of threads in the pool
    - Default value is the minimum of 32 or the number of processors in the system plus 4.
    - 5 workers are preserved for IO-bound tasks.  
- *initializer* is an optional callable, called at the start of each worker thread.
    - *initargs* is a tuple of arguments for *initializer*.  
Idle worker threads are reused before starting a new thread.

### 15.5.3 ProcessPoolExecutor <a name='Threads-ProcessPoolExecutor'></a>

`concurrent.futures.ThreadPoolExecutor` is a subclass of `Executor` that uses a process pool for creating and managing futures.

 &ensp;&ensp;&ensp;&ensp; `concurrent.futures.ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=())`
- *max_workers* must be 61 at most for Windows. If `None`, the default will be at most 61 even if more processors are present.  
- *mp_context* can be a multiprocessing context or `None`. If it's not specified or is `None`, the default multiprocessing context is used.

### 15.5.4 Future Objects <a name='Threads-Future-Objects'></a>

A `Future` encapsulates the execution of a callable. Futures are created by `Executor.submit()`. A `Future` object provides the following operations on its associated future:  

- `cancel()` - attempts to cancel the call. If the call is already started or finished, returns `False`.    Otherwise, the call is cancelled and returns `True`.
- `cancelled()` - returns `True` iff the call was successfully canceled.
- `running()` - returns `True` iff the call is currently executing and cannot be cancelled.
- `done()` - returns `True` iff the call was either cancelled or finished normally.
- `result(timeout=None)` - returns the value returned by the call.  
    - If the call hasn't completed by *timeout* seconds, raises a `concurrent.futures.TimeoutError`.
    - If the call is cancelled, raises a `CancelledError`.
- `exception(timeout=None)` - returns the exception raised by the call.
    - If the call hasn't completed by *timeout* seconds, raises a `concurrent.futures.TimeoutError`.
    - If the call is cancelled, raises a `CancelledError`.

### 15.5.5 Working with Concurrent Futures <a name='Threads-Working-with-Concurrent-Futures'></a>

In [ ]:
# 15.5.5.a Using a ProcessPoolExecutor and as_completed() to square integers 0-100
import concurrent.futures as futures

def sq(i):
  return f'{i} squared is {i**2}'

if __name__ == "__main__":
  with futures.ThreadPoolExecutor() as tpe:
    tasks = []
    for i in range(100):
      tasks.append(tpe.submit(sq,i))
    for task in futures.as_completed(tasks):
      print(task.result())

## 15.6 Python Interpreter (CPython) Limitations <a name='Threads-Python-Interpreter-(CPython)-Limitations'></a>

Python's standard interpreter, CPython, inhibits thread-level parallelism with the aid of a lock, the Global Interpreter Lock (GIL), that a thread must acquire in order to run and release in order to allow other threads to execute. [According to Adjitsara](https://realpython.com/python-gil/), preventing threads from executing in parallel allowed Python 2’s developers to implement a simple, efficient strategy for reclaiming memory used by objects that can no longer be referenced by a session’s variables. This policy was retained for Python 3, because all strategies that have been proposed for executing threads in parallel either require supporting libraries to be rewritten or degrade the performance of single-threaded Python programs. 

CPython’s use of the GIL has the following implications for exploiting concurrency and parallelism in Python programs:  
- To speed the execution of an application that spends considerable time waiting on external requests to complete,   use multithreading to implement concurrent execution.   This would be the advisable, for example, for a web application that uses multiple simultaneous HTTP requests to obtain data from external hosts. 
- Otherwise, adopt one of two strategies to implement the application:
    - If the application spends the bulk of its time processing data, use multiprocessing to parallelize the program’s operation.       This would be the advisable, for example, for an application that renders a video, where each cell can be rendered in isolation from all others.
    - Otherwise, use an implementation of a Python interpreter, like IronPython or PyPi, that does away with the GIL.       This would be the advisable , for example, for an algorithm that allocates repeated,       small computations to codes that then interact with one another, like matrix inversion.